In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [19]:
conn = sqlite3.connect('../laliga.sqlite')
df = pd.read_sql_query("SELECT * FROM Matches", conn)
display(df.head())
conn.close()

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2


In [22]:
# Separate the home score and the away score.
df[['home_score', 'away_score']] = df['score'].str.split(':', expand=True)
df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])

# Delete the matches that have not been played yet
df = df[(~df['home_score'].isnull()) | (~df['away_score'].isnull())]


# Define whether it is a home or away win.
df['result'] = df.apply(lambda row: 1 if row['home_score'] > row['away_score']
                                        else 2 if row['home_score'] < row['away_score']
                                        else 'X', axis=1)
display(df.head())

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,2,3,2
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,3,2,1
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,5,0,1
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,1,1,X
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,0,2,2


In [25]:
results = df.loc[:,'result']
predictors = df.loc[:,['home_team','away_team','season','matchday']]
display(predictors.head())

,home_team,away_team,season,matchday
0,Arenas Club,Athletic Madrid,1928-1929,1
1,Espanyol,Real Unión,1928-1929,1
2,Real Madrid,Catalunya,1928-1929,1
3,Donostia,Athletic,1928-1929,1
4,Racing,Barcelona,1928-1929,1


In [27]:
home_stats = pd.DataFrame()
away_stats = pd.DataFrame()

